In [131]:
import pandas as pd
import numpy as np
import nltk
import string
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, GPT2Tokenizer, AdamW, BertTokenizer,TFGPT2ForSequenceClassification
from transformers import TFBertForSequenceClassification,TFBertTokenizer,pipeline
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
import torch
from torch.optim import AdamW

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
!mkdir -p ~/.kaggle|
!cp kaggle.json ~/.kaggle/

/bin/bash: -c: line 2: syntax error: unexpected end of file


In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 19% 5.00M/25.7M [00:00<00:00, 36.0MB/s]
100% 25.7M/25.7M [00:00<00:00, 122MB/s] 


In [9]:
!unzip -q imdb-dataset-of-50k-movie-reviews.zip

In [141]:
main_data = pd.read_csv('/content/IMDB Dataset.csv')

In [142]:
main_data['sentiment'].replace({'positive':1,'negative':0},inplace=True)

In [87]:
data  = main_data.iloc[:2500]

In [83]:
data.shape

(2500, 2)

In [29]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [30]:
data['sentiment'].replace({'positive':1,'negative':0},inplace=True)

In [31]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [32]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [33]:
data.duplicated().sum()

0

In [34]:
sw = set(stopwords.words('english'))
stemmer = WordNetLemmatizer()

In [35]:
def fun2(x):
    # Remove HTML tags
    x = re.sub(r'<.*?>', '', x)

    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9]', ' ', x)

    # Replace single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Replace single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', ' ', text)

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # Tokenize, lowercase, remove stop words, and lemmatize
    tokens = word_tokenize(text)
    tokens = [i.lower() for i in tokens if i not in sw]
    tokens = [i for i in tokens if i not in string.punctuation]
    tokens = [stemmer.lemmatize(token) for token in tokens]
    x = ' '.join(tokens)

    return x

In [36]:
data['review'] = data['review'].apply(fun2)

In [37]:
data

,review,sentiment
0,one reviewer mentioned watching 1 oz episode h...,1
1,wonderful little production the filming techni...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake think zombie ...,0
4,petter mattei love time money visually stunnin...,1
...,...,...
2495,another great movie costa gavras it great pres...,0
2496,though structured totally different book tim k...,1
2497,handsome dashing british airline pilot george ...,1
2498,this film breech fine line satire silliness wh...,0


## Model Buinding

In [38]:
from transformers import AutoTokenizer
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

In [39]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [40]:
x = data['review']
y = data['sentiment']

In [41]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [42]:
def tokenize_function(examples):
    return tokenizer(examples, padding='max_length', truncation=True)

train_tokenized = x_train.apply(tokenize_function)
test_tokenized = x_test.apply(tokenize_function)

In [43]:
train_tokenized,y_train

(2055    [input_ids, token_type_ids, attention_mask]
 1961    [input_ids, token_type_ids, attention_mask]
 1864    [input_ids, token_type_ids, attention_mask]
 2326    [input_ids, token_type_ids, attention_mask]
 461     [input_ids, token_type_ids, attention_mask]
                            ...                     
 1638    [input_ids, token_type_ids, attention_mask]
 1095    [input_ids, token_type_ids, attention_mask]
 1130    [input_ids, token_type_ids, attention_mask]
 1294    [input_ids, token_type_ids, attention_mask]
 860     [input_ids, token_type_ids, attention_mask]
 Name: review, Length: 2000, dtype: object,
 2055    1
 1961    1
 1864    0
 2326    1
 461     1
        ..
 1638    0
 1095    0
 1130    0
 1294    1
 860     1
 Name: sentiment, Length: 2000, dtype: int64)

In [44]:
input_ids = np.array([item['input_ids'] for item in train_tokenized])
token_type_ids = np.array([item['token_type_ids'] for item in train_tokenized])
attention_mask = np.array([item['attention_mask'] for item in train_tokenized])

eval_input_ids = np.array([item['input_ids'] for item in test_tokenized])
eval_token_type_ids = np.array([item['token_type_ids'] for item in test_tokenized])
eval_attention_mask = np.array([item['attention_mask'] for item in test_tokenized])

y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

In [45]:
input_ids

array([[  101,  1122,  1121, ...,     0,     0,     0],
       [  101,  1142, 22302, ...,     0,     0,     0],
       [  101,   122,  1645, ...,     0,     0,     0],
       ...,
       [  101,  2640,  2458, ...,     0,     0,     0],
       [  101,  1142,  2523, ...,     0,     0,     0],
       [  101,  1142,  1707, ...,     0,     0,     0]])

In [68]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices(
    ({'input_ids': input_ids, 'token_type_ids': token_type_ids, 'attention_mask': attention_mask}, y_train_np)
).shuffle(len(input_ids)).batch(8)

eval_tf_dataset = tf.data.Dataset.from_tensor_slices(
    ({'input_ids': eval_input_ids, 'token_type_ids': eval_token_type_ids, 'attention_mask': eval_attention_mask}, y_test_np)
).batch(8)

In [58]:
train_tf_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(512,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int64, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [56]:
train_tf_dataset

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [48]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
#model = TFGPT2ForSequenceClassification.from_pretrained("gpt2",num_labels=2)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
last_layer = model.layers[-1]

# Change the activation function to sigmoid and set the number of neurons to 1
last_layer.activation = tf.keras.activations.softmax
last_layer.units = 2

In [50]:
# Assuming model is your TFAutoModelForSequenceClassification model
last_layer = model.layers[-1]

# Check the activation function
activation_function = last_layer.activation.__name__
print(f"The activation function of the last layer is: {activation_function}")

# Check the number of neurons
num_neurons = last_layer.units
print(f"The number of neurons in the last layer is: {num_neurons}")

The activation function of the last layer is: softmax
The number of neurons in the last layer is: 2


In [51]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108311810 (413.18 MB)
Trainable params: 108311810 (413.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [53]:
model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=10)

Epoch 1/10
250/250 [==============================] - 305s 1s/step - loss: 0.6801 - accuracy: 0.5575 - val_loss: 0.4996 - val_accuracy: 0.7900
Epoch 2/10
250/250 [==============================] - 259s 1s/step - loss: 0.4186 - accuracy: 0.8190 - val_loss: 0.3622 - val_accuracy: 0.8280
Epoch 3/10
250/250 [==============================] - 258s 1s/step - loss: 0.2384 - accuracy: 0.9105 - val_loss: 0.3588 - val_accuracy: 0.8640
Epoch 4/10
250/250 [==============================] - 259s 1s/step - loss: 0.1418 - accuracy: 0.9515 - val_loss: 0.3542 - val_accuracy: 0.8720
Epoch 5/10
250/250 [==============================] - 258s 1s/step - loss: 0.0597 - accuracy: 0.9860 - val_loss: 0.9684 - val_accuracy: 0.8300
Epoch 6/10
250/250 [==============================] - 259s 1s/step - loss: 0.0465 - accuracy: 0.9885 - val_loss: 0.5897 - val_accuracy: 0.8660
Epoch 7/10
250/250 [==============================] - 278s 1s/step - loss: 0.0618 - accuracy: 0.9795 - val_loss: 0.5519 - val_accuracy: 0.8600

In [60]:
model.save('bert_fine_tune', save_format='tf')

In [69]:
model.evaluate(eval_tf_dataset)

63/63 [==============================] - 22s 351ms/step - loss: 0.8460 - accuracy: 0.8500


[0.8459591865539551, 0.8500000238418579]

In [89]:
test_data

,review,sentiment
2500,But I can't say how I really feel about this p...,0
2501,From the opening scene aboard a crowded train ...,1
2502,I have seen a couple movies on eating disorder...,1
2503,H.O.T.S. is not for those that want hardcore p...,1
2504,THAT'S certainly a strange way to promote a fi...,0
...,...,...
2595,"Okay, it's too long and it's too satisfied wit...",1
2596,"Unbelievable. ""Philosophy"". ""Depth"". ""Genius""....",0
2597,"Well, it's yet again a film that plays with yo...",1
2598,Erika Kohut is a woman with deep sexual proble...,1


In [157]:
test = main_data.iloc[3002]

In [158]:
test_x = test['review']
test_y = test['sentiment']

In [159]:
test_x

"For those not in the know, the Asterix books are a hugely successful series of comic books about a village of indomitable Gauls who resist Caesar's invasion thanks to a magic potion that renders them invulnerable supermen. There have been several animated features (only one of them, The Twelve Tasks of Asterix really capturing the wit and spirit of the books despite being an original screen story) before a perfectly cast Christian Clavier and Gerard Depardieu took the lead roles in two live action adaptations that proved colossally successful throughout Europe but made no impression whatsoever in the English-speaking world. <br /><br />The uncut French version is great fun, but sadly does not appear to be available in a version with English subtitles outside of the UK DVD. While there's still no sign of a US theatrical or DVD release, the Miramax version of Asterix et Obelix: Mission Cleopatre is also on that DVD (and has played on UK TV), and you'll never guess what - it's been compl

In [160]:
test_tokenized = tokenizer(test_x, padding='max_length', truncation=True)


In [161]:
input_ids = tf.constant([test_tokenized['input_ids']])
token_type_ids = tf.constant([test_tokenized['token_type_ids']])
attention_mask = tf.constant([test_tokenized['attention_mask']])

In [162]:
input_data = {
    'input_ids': input_ids,
    'token_type_ids': token_type_ids,
    'attention_mask': attention_mask
}

In [163]:
#input_data

In [164]:
pred = model.predict(input_data)

1/1 [==============================] - 0s 95ms/step


In [165]:
pred[0][0]

array([0.27388328, 0.7261167 ], dtype=float32)

In [166]:
value = np.argmax(pred[0][0])

In [167]:
value,test_y

(1, 1)

### Using Pipeline

In [171]:
sentiment_analysis = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Example usage:
text = "you are so very bad"

# Make predictions
result = sentiment_analysis(text)

# Print the result
print(result)

[{'label': 'LABEL_0', 'score': 0.6803487539291382}]


In [134]:
#sentiment_analysis.save_pretrained('pipeline')